## Import libraries

In [8]:
import flodym as fd
from flodym import ExcelDimensionReader, DimensionDefinition
from flodym import ExcelParameterReader, ParameterDefinition
import numpy as np
import flodym.export as fde

## Definitions

In [9]:
dimension_definitions = [
    fd.DimensionDefinition(letter="t", name="time", dtype=int),
    fd.DimensionDefinition(letter="c", name="time_construction", dtype=int),
    fd.DimensionDefinition(letter="b", name="building sector", dtype=str),
    fd.DimensionDefinition(letter="s", name="scenario", dtype=str),
    fd.DimensionDefinition(letter="p", name="product", dtype=str),
]

parameter_definitions = [
    fd.ParameterDefinition(name="building area per capita", dim_letters=("t","s")),
    fd.ParameterDefinition(name="population", dim_letters=("t",)),
    fd.ParameterDefinition(name="concrete mass per area", dim_letters=("t","p","s")),
    fd.ParameterDefinition(name="concrete mass per area historic", dim_letters=("c","p","s")),
    fd.ParameterDefinition(name="product share", dim_letters=("t","p")),
    fd.ParameterDefinition(name="stock product age cohort", dim_letters=("t","c","s","p")),
    fd.ParameterDefinition(name="product lifetimes", dim_letters=("c","s")),
]

In [10]:
process_names = [
    "sysenv",
    "use",
    "waste"
]

In [11]:
flow_definitions = [
    fd.FlowDefinition(from_process_name="sysenv", to_process_name="use", dim_letters=("c", "s", "p")),
    fd.FlowDefinition(from_process_name="use", to_process_name="waste", dim_letters=("c", "s")),
]

stock_definitions = [
    fd.StockDefinition(
        name="use",
        process="use",
        dim_letters=("c", "s", "p"),
        time_letter="c",
        subclass=fd.StockDrivenDSM,
        lifetime_model_class=fd.NormalLifetime,
    ),
        fd.StockDefinition(
        name="waste",
        process="waste",
        dim_letters=("c", "s"),
        time_letter="c",
        subclass=fd.SimpleFlowDrivenStock, # stock as the accumulation of inflow
    ),
]

In [12]:
mfa_definition = fd.MFADefinition(
    dimensions=dimension_definitions,
    parameters=parameter_definitions,
    processes=process_names,
    flows=flow_definitions,
    stocks=stock_definitions,
)

## Data import

In [13]:
dimension_file = "../Data/dimension.xlsx"
dimension_files = {d.name: dimension_file for d in dimension_definitions}
dimension_sheets = {d.name: d.name for d in dimension_definitions}
reader = ExcelDimensionReader(
    dimension_files=dimension_files,
    dimension_sheets=dimension_sheets,
)

dims = reader.read_dimensions(dimension_definitions)

parameter_file = "../Data/parameter.xlsx"
parameter_files = {p.name: parameter_file for p in parameter_definitions}
parameter_sheets = {p.name: p.name for p in parameter_definitions}
reader = ExcelParameterReader(
    parameter_files=parameter_files,
    parameter_sheets=parameter_sheets,
    allow_missing_values=True,
    allow_extra_values=True,
)

parameters = reader.read_parameters(parameter_definitions=parameter_definitions, dims=dims)


## Define MFA model

In [14]:
class ConcreteMFA(fd.MFASystem):
    def compute(self):
        future_concrete_product_stock = self.parameters["population"] * self.parameters["building area per capita"] * \
                                        self.parameters["concrete mass per area"] * self.parameters["product share"]
        # print(f'future_concrete_product_stock: {future_concrete_product_stock}')

        # set the lifetime model parameters for stocks["use"]
        self.stocks["use"].lifetime_model.set_prms(
            mean=self.parameters["product lifetimes"],
            std=0.3 * parameters["product lifetimes"],  # assume std = 30% of mean
        )
        # build survival function for stocks["use"] using sf function (numpy array, ["c", "C", "s", "b", "p"])
        survival_historic_values = self.stocks["use"].lifetime_model.sf
        # print(type(survival_historic_values))
        # print(survival_historic_values.shape)

        # create FlodymArray survival_historic for survival function with correct dimensions
        c2 = fd.Dimension(name="time_construction2", letter="C", items=self.dims["c"].items)
        dims = fd.DimensionSet(dim_list=[c2] + self.stocks["use"].dims.dim_list)
        # as the stock composition is only available for 2015, slice the survival histroric to keep only C=2015
        survival_historic = fd.FlodymArray(values=survival_historic_values, dims=dims)[{"C": 2015}]

        # calculate the historical inflow from the stock composition and survival function for 2015
        # will get warning message for years later than 2015 where survival function = 0
        # but this will not affect the calculation as the stock after 2015 will be overwritten with predicted data later
        # this includes nan value => replace with 0
        inflow_historic = self.parameters["stock product age cohort"][{"t": 2015}] / survival_historic
        inflow_historic_values = inflow_historic.values
        inflow_historic_values_where_nan = np.isnan(inflow_historic_values)
        inflow_historic_values[inflow_historic_values_where_nan] = 0

        inflow_historic_nonan = fd.FlodymArray(values=inflow_historic_values, dims=inflow_historic.dims)*self.parameters["concrete mass per area historic"]

        # feed the historical inflow to an inflow-driven DSM
        dsm_historic = fd.InflowDrivenDSM(
            dims=inflow_historic_nonan.dims,
            lifetime_model=self.stocks["use"].lifetime_model,
            time_letter="c",
        )
        dsm_historic.inflow[...] = inflow_historic_nonan
        dsm_historic.compute()

        # print(f'historic stock values from DSM: {dsm_historic.stock}')

        # change the use stock with historical stock data from DSM
        self.stocks["use"].stock[...] = dsm_historic.stock

        # overwrite future development with predicted stock
        self.stocks["use"].stock[{"c": self.dims["t"]}] = future_concrete_product_stock

        # print(f'future stock values: {self.stocks["use"].stock}')

        # compute use stock inflow and outflow
        self.stocks["use"].compute()

        total_outflow = self.stocks["use"].outflow



        # define envsys => use flow
        self.flows["sysenv => use"][...] = self.stocks["use"].inflow

        self.flows["use => waste"][...] = total_outflow

        self.stocks["waste"].inflow[...] = self.flows["use => waste"][...]
        self.stocks["waste"].compute()
        # print(f'dimensions use_outflow: {self.flows["use => waste"].dims}')
        # print(f'use_outflow values: {self.flows["use => waste"].values}')


In [16]:
# create an instance of ConcreteMFA using excel data
ConcreteMFA_FR = ConcreteMFA.from_excel(
    definition=mfa_definition,
    dimension_files=dimension_files,
    parameter_files=parameter_files,
    dimension_sheets=dimension_sheets,
    parameter_sheets=parameter_sheets,
    allow_extra_parameter_values=True,
    allow_missing_parameter_values=True,
)

In [17]:
ConcreteMFA_FR.compute() #execute

/Users/zhangyiwen/anaconda3/lib/python3.11/site-packages/flodym/flodym_arrays.py:349: RuntimeWarning: divide by zero encountered in divide
  1.0 / other.values,


In [20]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

ConcreteMFA_FR.check_mass_balance()

INFO:root:Checking mass balance of ConcreteMFA object...
INFO:root:Success - Mass balance is consistent!


In [21]:
use_outflow=ConcreteMFA_FR.stocks["use"].outflow[...]
use_stock=ConcreteMFA_FR.stocks["use"].stock[...]
sys_inflow=ConcreteMFA_FR.stocks["use"].inflow[...]

In [ ]:
use_outflow.dims.dim_list

year_waste = fd.Dimension(name="year_waste", letter="t", items=range(2015, 2100,1))

use_outflow_BAU = use_outflow["BAU"]
use_outflow_CE = use_outflow["CE"]

print(use_outflow_BAU)


use_outflow_BAU_t = use_outflow_BAU[{"c": year_waste}]
print(use_outflow_BAU_t)

use_outflow_BAU_t_waste = use_outflow_BAU_t * self.parameters["waste share"]

FlodymArray 'use_outflow' with dims (c,p) and shape (201, 17);
Values:
[[   0.            0.            0.         ...    0.
     0.            0.        ]
 [   0.            0.            0.         ...    0.
     0.            0.        ]
 [   0.            0.            0.         ...    0.
     0.            0.        ]
 ...
 [   0.         3855.90854748    0.         ...  186.51560499
  1498.33393657  498.68574247]
 [   0.         3997.27766886    0.         ...  187.36742235
  1505.22343762  501.08901538]
 [   0.         4148.63063921    0.         ...  188.65396863
  1514.94145592  505.16660349]]
FlodymArray 'use_outflow' with dims (t,p) and shape (85, 17);
Values:
[[    0.         12356.62274982     0.         ...    93.34803432
    706.79464996   268.10350068]
 [    0.         12612.50343571     0.         ...   104.83027756
    788.13729793   307.15388863]
 [    0.         12612.40339777     0.         ...   110.62264009
    830.80887318   325.56143392]
 ...
 [    0.         

In [ ]:
fig = fde.PlotlySankeyPlotter(mfa=ConcreteMFA_FR, exclude_processes=[]).plot()
fig.show()